In [ ]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import json

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Acceder a la variable de entorno PYTHON_URL_BACK
PYTHON_URL_BACK = os.getenv("PYTHON_URL_BACK")

def obtener_detalles_clientes():
    response = requests.get(f"{PYTHON_URL_BACK}/user")
    if response.status_code != 200:
        return None
    
    users = response.json()
    user_df = pd.DataFrame(users)
    user_df['full_name'] = user_df['firstName'] + ' ' + user_df['lastName']
    
    return user_df[['id', 'full_name']]

def obtener_promedio_diners_venta():
    response = requests.get(f"{PYTHON_URL_BACK}/orders")
    if response.status_code != 200:
        return []
    
    orders = response.json()
    df_orders = pd.DataFrame(orders)
    
    df_orders['total'] = pd.to_numeric(df_orders['total'], errors='coerce')
    df_orders = df_orders.dropna(subset=['total'])
    
    df_sales = df_orders.groupby('userId').agg(
        Diners_per_Venda=('total', 'mean'),
        Ventas=('total', 'count'),
        Total_gastado=('total', 'sum')
    ).reset_index()
    
    df_sales['Diners_per_Venda'] = df_sales['Diners_per_Venda'].round(2)
    
    user_df = obtener_detalles_clientes()
    
    df_sales = df_sales.merge(user_df, how='left', left_on='userId', right_on='id')
    df_sales = df_sales.drop(columns=['id'])
    
    df_sales['Moneda'] = '€'
    df_sales['ID'] = df_sales['userId'].apply(lambda x: f"{x:02d}" if x < 10 else str(x))
    
    df_sales = df_sales[['ID', 'full_name', 'Ventas', 'Total_gastado', 'Diners_per_Venda', 'Moneda']]
    df_sales.columns = ['ID', 'Clients', 'Ventas', 'Diners', 'Diners/Venda', 'Moneda']
    
    return df_sales.to_dict('records')

if __name__ == "__main__":
    resultado = obtener_promedio_diners_venta()
    print(json.dumps(resultado))


Datos de ventas por cliente:
   ID      Clients  Ventas  Diners  Diners/Venda Moneda
0  01     John Doe       1   75.98         75.98      €
1  02   Jane Smith       1   89.99         89.99      €
2  03  Alice Brown       1   70.98         70.98      €
3  05          a a       3  373.90        124.63      €
